### Importing Libraries 

In [2]:
from influxdb_client import InfluxDBClient, WriteOptions
from influxdb_client.client.write_api import SYNCHRONOUS
import influxdb_client
import pandas as pd
import random
import numpy as np
from datetime import datetime

### Connection 

In [3]:
# Configuration
token = "pee4Yh4gB8HFYWIyxNpgw0lacZVlJAaNpbpWHdT-49l1Y3ZhLuI4N3k9Ec6gC4LGc7IIi8kfOMlD41iSqI__Xg=="
org = "Project_Big_Data"
bucket = "Time_Bucket"
url = "http://localhost:8086"

#Instantiate the client
client = InfluxDBClient(url=url, token=token, org=org)

#Instantiating  a write client
write_api = client.write_api(write_options=SYNCHRONOUS)

#Creating a point object
p = influxdb_client.Point("temperature_measurement").tag("location", "Prague").field("temperature", 25.3)
try: 
 write_api.write(bucket=bucket, org=org, record=p)
 print("Succefully Data has been inserted into the DB")
except:
 print("Some Error occurred during Data injection")


Succefully Data has been inserted into the DB


### Query data from InfluxDB

In [4]:
query_api = client.query_api()

query = 'from(bucket:"Time_Bucket")\
|> range(start: -20m)\
|> filter(fn:(r) => r._measurement == "temperature_measurement")\
|> filter(fn:(r) => r.location == "Prague")\
|> filter(fn:(r) => r._field == "temperature")'

result = query_api.query(org=org, query=query)

results = []
for table in result:
  for record in table.records:
    results.append(({'Measurement': record.get_measurement()}, {'Field': record.get_field()},  {'value':record.get_value()},{'Time': record.get_time()}))
    # , ,, ))

print(results)


[({'Measurement': 'temperature_measurement'}, {'Field': 'temperature'}, {'value': 25.3}, {'Time': datetime.datetime(2024, 5, 17, 13, 1, 11, 889413, tzinfo=tzutc())})]


### Performing Insertion and Querying on 100 data points 

Considering an application which consists of weather data and preserves the data of Temperature, Humidity, Pressure and Weather daily conditions in the city of KIEL. For this demo purpose, here random values have been generated to fill in the data which can be in practical real application can be collected with the help of sensors or other weather forecasting platform.

### Generating Measurements

### 1. Temperature

In [44]:
# generating random temperature values
def generate_temperature_data(num_measurements, base_temperature=20, variation=5):
    np.random.seed(0)  # For reproducibility
    temperatures = base_temperature + variation * np.random.randn(num_measurements)
    return temperatures

randomly_generated_temp = generate_temperature_data(num_measurements=100)

def generate_temp_measurement(): 
    point = influxdb_client.Point(measurement_name="temperature_table")
    tags = {"city":"Kiel"}
    fields = {"value":random.choice(randomly_generated_temp)}
    for tag_key, tag_value in tags.items():
        point.tag(tag_key, tag_value)
    for filed_key, field_value in fields.items():
        point.field(filed_key, field_value)

    return point


# Inserting 100 points 
error_occured = False
for i in range(100):
    measurement = generate_temp_measurement()
    try:
     write_api.write(bucket="Weather_Bucket", record = measurement)
    except Exception as e: 
       error_occured = True
       print("Error in inserting the data:", e)
       
if not error_occured: 
   print("Successful insertion of data ")   

Successful insertion of data 


### 2. Pressure

In [46]:
# Function to generate random pressure data
def generate_pressure_data(num_measurements, base_pressure=1013, variation=10):
    pressures = base_pressure + variation * np.random.randn(num_measurements)
    return pressures

pressures = generate_pressure_data(100)

def generate_pressure_measurement(): 
    point = influxdb_client.Point(measurement_name="pressure_table")
    tags = {"city":"Kiel"}
    fields = {"value":random.choice(pressures)}
    for tag_key, tag_value in tags.items():
        point.tag(tag_key, tag_value)
    for filed_key, field_value in fields.items():
        point.field(filed_key, field_value)

    return point

# Inserting 100 points 
error_occured = False
for i in range(100):
    measurement = generate_pressure_measurement()
    try:
     write_api.write(bucket="Weather_Bucket", record = measurement)
    except Exception as e: 
       error_occured = True
       print("Error in inserting the data:", e)
       
if not error_occured: 
   print("Successful insertion of data ") 

Successful insertion of data 


### 3. Humidity

In [47]:
def generate_humidity_data(num_measurements, base_humidity=50, variation=20):
    humidities = base_humidity + variation * np.random.randn(num_measurements)
    humidities = np.clip(humidities, 0, 100)  # Ensure humidity is between 0 and 100
    return humidities

humidities = generate_humidity_data(100)

def generate_humidity_measurement(): 
    point = influxdb_client.Point(measurement_name="humidity_table")
    tags = {"city":"Kiel"}
    fields = {"value":random.choice(humidities)}
    for tag_key, tag_value in tags.items():
        point.tag(tag_key, tag_value)
    for filed_key, field_value in fields.items():
        point.field(filed_key, field_value)

    return point

# Inserting 100 points 
error_occured = False
for i in range(100):
    measurement = generate_humidity_measurement()
    try:
     write_api.write(bucket="Weather_Bucket", record = measurement)
    except Exception as e: 
       error_occured = True
       print("Error in inserting the data:", e)
       
if not error_occured: 
   print("Successful insertion of data ") 



Successful insertion of data 


### Querying Data

Temperature

In [48]:
query_api = client.query_api()


weather_query_temperature =  'from(bucket:"Weather_Bucket")\
|> range(start: -1d)\
|> filter(fn:(r) => r._measurement == "temperature_table")'
                
result = query_api.query(query=weather_query_temperature, org = org)

# Process the results
results_temperature = []
temp_dictionary = {}
temp_record_values = []
for table in result:
  print("Number of Records Retrived for Temperature",len(table.records))
  for record in table.records:
    temp_record_values.append( record.get_value())
    results_temperature.append(({'Measurement': record.get_measurement()}, {'Field': record.get_field()},  {'value':record.get_value()},{'Time': record.get_time()}))

print(results)
temp_dictionary['temperature'] = temp_record_values
print("Dictionary Created: ", temp_dictionary)

Number of Records Retrived for Temperature 100
[({'Measurement': 'humidity_record'}, {'Field': 'value'}, {'value': 59.87483554698377}, {'Time': datetime.datetime(2024, 5, 17, 13, 30, 5, 244672, tzinfo=tzutc())}), ({'Measurement': 'humidity_record'}, {'Field': 'value'}, {'value': 54.643620724005515}, {'Time': datetime.datetime(2024, 5, 17, 13, 30, 5, 257492, tzinfo=tzutc())}), ({'Measurement': 'humidity_record'}, {'Field': 'value'}, {'value': 43.08036448601227}, {'Time': datetime.datetime(2024, 5, 17, 13, 30, 5, 261919, tzinfo=tzutc())}), ({'Measurement': 'humidity_record'}, {'Field': 'value'}, {'value': 89.11824616501389}, {'Time': datetime.datetime(2024, 5, 17, 13, 30, 5, 266990, tzinfo=tzutc())}), ({'Measurement': 'humidity_record'}, {'Field': 'value'}, {'value': 20.67151344394972}, {'Time': datetime.datetime(2024, 5, 17, 13, 30, 5, 270548, tzinfo=tzutc())}), ({'Measurement': 'humidity_record'}, {'Field': 'value'}, {'value': 47.67792121931267}, {'Time': datetime.datetime(2024, 5, 17,

Pressure

In [49]:
query_api = client.query_api()


weather_query_pressure =  'from(bucket:"Weather_Bucket")\
|> range(start: -1d)\
|> filter(fn:(r) => r._measurement == "pressure_table")'
                
result = query_api.query(query=weather_query_pressure, org = org)

# Process the results
results_pressure = []
pressure_dictionary = {}
pressure_record_values = []

for table in result:
  print("Number of Records Retrived for Pressure",len(table.records))
  for record in table.records:
    pressure_record_values.append( record.get_value())
    results.append(({'Measurement': record.get_measurement()}, {'Field': record.get_field()},  {'value':record.get_value()},{'Time': record.get_time()}))

print(results)
pressure_dictionary['pressure'] = pressure_record_values

print("Dictionary created:", pressure_dictionary)

Number of Records Retrived for Pressure 100
[({'Measurement': 'humidity_record'}, {'Field': 'value'}, {'value': 59.87483554698377}, {'Time': datetime.datetime(2024, 5, 17, 13, 30, 5, 244672, tzinfo=tzutc())}), ({'Measurement': 'humidity_record'}, {'Field': 'value'}, {'value': 54.643620724005515}, {'Time': datetime.datetime(2024, 5, 17, 13, 30, 5, 257492, tzinfo=tzutc())}), ({'Measurement': 'humidity_record'}, {'Field': 'value'}, {'value': 43.08036448601227}, {'Time': datetime.datetime(2024, 5, 17, 13, 30, 5, 261919, tzinfo=tzutc())}), ({'Measurement': 'humidity_record'}, {'Field': 'value'}, {'value': 89.11824616501389}, {'Time': datetime.datetime(2024, 5, 17, 13, 30, 5, 266990, tzinfo=tzutc())}), ({'Measurement': 'humidity_record'}, {'Field': 'value'}, {'value': 20.67151344394972}, {'Time': datetime.datetime(2024, 5, 17, 13, 30, 5, 270548, tzinfo=tzutc())}), ({'Measurement': 'humidity_record'}, {'Field': 'value'}, {'value': 47.67792121931267}, {'Time': datetime.datetime(2024, 5, 17, 13

Humdity

In [50]:
query_api = client.query_api()


weather_query_humidity =  'from(bucket:"Weather_Bucket")\
|> range(start: -1d)\
|> filter(fn:(r) => r._measurement == "humidity_table")'
                
result = query_api.query(query=weather_query_humidity, org = org)

# Process the results
results = []
humidity_dictionary = {}
humidity_record_values = []
for table in result:
  print("Number of Records Retrived for Humidity",len(table.records))
  for record in table.records:
    humidity_record_values.append( record.get_value())
    results.append(({'Measurement': record.get_measurement()}, {'Field': record.get_field()},  {'value':record.get_value()},{'Time': record.get_time()}))

print(results)
humidity_dictionary['humidity'] = humidity_record_values

print("Dictionary created:", humidity_dictionary)

Number of Records Retrived for Humidity 100
[({'Measurement': 'humidity_table'}, {'Field': 'value'}, {'value': 39.14277047966564}, {'Time': datetime.datetime(2024, 5, 17, 14, 48, 2, 561005, tzinfo=tzutc())}), ({'Measurement': 'humidity_table'}, {'Field': 'value'}, {'value': 68.20357816185184}, {'Time': datetime.datetime(2024, 5, 17, 14, 48, 2, 572707, tzinfo=tzutc())}), ({'Measurement': 'humidity_table'}, {'Field': 'value'}, {'value': 58.32100092522851}, {'Time': datetime.datetime(2024, 5, 17, 14, 48, 2, 578684, tzinfo=tzutc())}), ({'Measurement': 'humidity_table'}, {'Field': 'value'}, {'value': 37.43824880716842}, {'Time': datetime.datetime(2024, 5, 17, 14, 48, 2, 582704, tzinfo=tzutc())}), ({'Measurement': 'humidity_table'}, {'Field': 'value'}, {'value': 20.67151344394972}, {'Time': datetime.datetime(2024, 5, 17, 14, 48, 2, 585704, tzinfo=tzutc())}), ({'Measurement': 'humidity_table'}, {'Field': 'value'}, {'value': 43.6134316570981}, {'Time': datetime.datetime(2024, 5, 17, 14, 48, 2,

### Pandas Dataframe

In [52]:
# Create individual DataFrames
df1 = pd.DataFrame(temp_dictionary)
df2 = pd.DataFrame(pressure_dictionary)
df3 = pd.DataFrame(humidity_dictionary)


# Concatenate DataFrames along columns
weather_df = pd.concat([df1, df2, df3], axis=1)

weather_df

,temperature,pressure,humidity
0,26.011899,1032.100650,39.142770
1,11.930511,1016.187277,68.203578
2,7.235051,1032.436212,58.321001
3,19.483906,1012.015475,37.438249
4,14.174251,1005.604370,20.671513
...,...,...,...
95,25.272259,1004.735615,38.484241
96,18.974209,1021.235042,20.671513
97,13.825871,1001.268766,16.479924
98,18.260439,1019.252315,51.768442


Thus this dataframe could then be use to generate data analysis on the basis of time and other factors of a particular city.

<i>End of File</i>